In [1]:
import os, json, glob, csv
from typing import Dict, Any, List

In [2]:
# --- 매핑: 코드 -> {label, group, type} --------------------------------------
SUPPORT_COND_MAP: Dict[str, Dict[str, str]] = {
    # 식별
    "서비스ID": {"label": "서비스ID", "group": "식별자", "type": "string"},
    "서비스명": {"label": "서비스명", "group": "식별자", "type": "string"},

    # JA01xx 성별/연령
    "JA0101": {"label": "남성", "group": "성별/연령", "type": "string"},
    "JA0102": {"label": "여성", "group": "성별/연령", "type": "string"},
    "JA0110": {"label": "대상연령(시작, 만 나이)", "group": "성별/연령", "type": "integer"},
    "JA0111": {"label": "대상연령(종료, 만 나이)", "group": "성별/연령", "type": "integer"},

    # JA02xx 소득
    "JA0201": {"label": "중위소득 0~50%", "group": "소득", "type": "string"},
    "JA0202": {"label": "중위소득 51~75%", "group": "소득", "type": "string"},
    "JA0203": {"label": "중위소득 76~100%", "group": "소득", "type": "string"},
    "JA0204": {"label": "중위소득 101~200%", "group": "소득", "type": "string"},
    "JA0205": {"label": "중위소득 200% 초과", "group": "소득", "type": "string"},

    # JA03xx 개인대상
    "JA0301": {"label": "예비부모/난임", "group": "개인대상", "type": "string"},
    "JA0302": {"label": "임산부", "group": "개인대상", "type": "string"},
    "JA0303": {"label": "출산/입양", "group": "개인대상", "type": "string"},
    "JA0313": {"label": "농업인", "group": "개인대상", "type": "string"},
    "JA0314": {"label": "어업인", "group": "개인대상", "type": "string"},
    "JA0315": {"label": "축산업인", "group": "개인대상", "type": "string"},
    "JA0316": {"label": "임업인", "group": "개인대상", "type": "string"},
    "JA0317": {"label": "초등학생", "group": "개인대상", "type": "string"},
    "JA0318": {"label": "중학생", "group": "개인대상", "type": "string"},
    "JA0319": {"label": "고등학생", "group": "개인대상", "type": "string"},
    "JA0320": {"label": "대학생/대학원생", "group": "개인대상", "type": "string"},
    "JA0322": {"label": "개인대상 해당사항없음", "group": "개인대상", "type": "string"},
    "JA0326": {"label": "근로자/직장인", "group": "개인대상", "type": "string"},
    "JA0327": {"label": "구직자/실업자", "group": "개인대상", "type": "string"},
    "JA0328": {"label": "장애인", "group": "개인대상", "type": "string"},
    "JA0329": {"label": "국가보훈대상자", "group": "개인대상", "type": "string"},
    "JA0330": {"label": "질병/질환자", "group": "개인대상", "type": "string"},

    # JA04xx 가구특성
    "JA0401": {"label": "다문화가족", "group": "가구특성", "type": "string"},
    "JA0402": {"label": "북한이탈주민", "group": "가구특성", "type": "string"},
    "JA0403": {"label": "한부모가정/조손가정", "group": "가구특성", "type": "string"},
    "JA0404": {"label": "1인가구", "group": "가구특성", "type": "string"},
    "JA0410": {"label": "가구특성 해당사항없음", "group": "가구특성", "type": "string"},
    "JA0411": {"label": "다자녀가구", "group": "가구특성", "type": "string"},
    "JA0412": {"label": "무주택세대", "group": "가구특성", "type": "string"},
    "JA0413": {"label": "신규전입", "group": "가구특성", "type": "string"},
    "JA0414": {"label": "확대가족", "group": "가구특성", "type": "string"},

    # JA11xx 창업/영업상태
    "JA1101": {"label": "예비창업자", "group": "창업/영업상태", "type": "string"},
    "JA1102": {"label": "영업중", "group": "창업/영업상태", "type": "string"},
    "JA1103": {"label": "생계곤란/폐업예정자", "group": "창업/영업상태", "type": "string"},

    # JA12xx 개인업종
    "JA1201": {"label": "음식점업", "group": "개인업종", "type": "string"},
    "JA1202": {"label": "제조업", "group": "개인업종", "type": "string"},
    "JA1299": {"label": "기타업종(개인)", "group": "개인업종", "type": "string"},

    # JA21xx 조직유형
    "JA2101": {"label": "중소기업", "group": "조직유형", "type": "string"},
    "JA2102": {"label": "사회복지시설", "group": "조직유형", "type": "string"},
    "JA2103": {"label": "기관/단체", "group": "조직유형", "type": "string"},

    # JA22xx 기업업종
    "JA2201": {"label": "제조업(기업/기관)", "group": "기업업종", "type": "string"},
    "JA2202": {"label": "농업·임업 및 어업(기업/기관)", "group": "기업업종", "type": "string"},
    "JA2203": {"label": "정보통신업(기업/기관)", "group": "기업업종", "type": "string"},
    "JA2299": {"label": "기타업종(기업/기관)", "group": "기업업종", "type": "string"},
}
CODE_TO_GROUP = {k: v["group"] for k, v in SUPPORT_COND_MAP.items()}
CODE_TO_TYPE  = {k: v["type"]  for k, v in SUPPORT_COND_MAP.items()}
CODE_TO_LABEL = {k: v["label"] for k, v in SUPPORT_COND_MAP.items()}

In [3]:
# --- 정규화 ---------------------------------------------------------------
def normalize_support_conditions(rec: Dict[str, Any]) -> Dict[str, Any]:
    out: Dict[str, Any] = {}
    # 식별자 유지
    for ident in ("서비스ID", "서비스명"):
        if ident in rec:
            out[ident] = rec[ident]

    for code, meta in SUPPORT_COND_MAP.items():
        if code in ("서비스ID", "서비스명"):
            continue
        if code not in rec:
            continue

        group = meta["group"]; label = meta["label"]; typ = meta["type"]
        key = f"{group}.{label}"
        val = rec[code]

        if typ == "integer":
            out[key] = None if val in (None, "") else int(val)
        else:
            out[key] = (str(val).upper() == "Y") if val is not None else False
    return out

# --- 파일 입출력 -----------------------------------------------------------
def load_json_any(path: str):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def extract_records(obj) -> List[Dict[str, Any]]:
    """
    supportConditions 응답 다양한 형태 지원:
    - {"data": [ {...}, {...} ]}
    - [ {...}, {...} ]
    - 단일 객체 { ... }
    """
    if isinstance(obj, dict):
        if isinstance(obj.get("data"), list):
            return obj["data"]
        # 다른 키에 리스트가 있을 때
        for v in obj.values():
            if isinstance(v, list) and v and isinstance(v[0], dict) and any(k.startswith("JA") for k in v[0].keys()):
                return v
        # 단일 객체
        if any(k.startswith("JA") for k in obj.keys()):
            return [obj]
        return []
    elif isinstance(obj, list):
        return obj
    return []

def save_json(path: str, rows: List[Dict[str, Any]]):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(rows, f, ensure_ascii=False, indent=2)

def save_csv(path: str, rows: List[Dict[str, Any]]):
    if not rows:
        return
    # 헤더: 식별자 먼저, 나머지 알파벳순
    all_keys = set().union(*[r.keys() for r in rows])
    fields = ["서비스ID", "서비스명"] + sorted(k for k in all_keys if k not in ("서비스ID", "서비스명"))
    with open(path, "w", newline="", encoding="utf-8-sig") as f:
        w = csv.DictWriter(f, fieldnames=fields)
        w.writeheader()
        w.writerows(rows)

def main():
    json_files = sorted(glob.glob("*.json"))
    if not json_files:
        print("⚠️ 현재 폴더에 .json 파일이 없습니다.")
        return

    combined: List[Dict[str, Any]] = []
    for fp in json_files:
        try:
            obj = load_json_any(fp)
            recs = extract_records(obj)
            if not recs:
                print(f"- 건너뜀(레코드 없음): {fp}")
                continue
            rows = [normalize_support_conditions(r) for r in recs]

            stem = os.path.splitext(os.path.basename(fp))[0]
            out_json = f"{stem}_normalized.json"
            out_csv  = f"{stem}_normalized.csv"
            save_json(out_json, rows)
            save_csv(out_csv, rows)
            combined.extend(rows)
            print(f"✅ {fp} → {out_json}, {out_csv} (행 {len(rows)})")
        except Exception as e:
            print(f"❌ 실패: {fp} ({e})")

    if combined:
        save_json("support_conditions_all_normalized.json", combined)
        save_csv("support_conditions_all_normalized.csv", combined)
        print(f"📦 합본 저장: support_conditions_all_normalized.(json/csv) — 총 {len(combined)}행")

In [4]:
if __name__ == "__main__":
    main()

✅ service_detail_000000465790.json → service_detail_000000465790_normalized.json, service_detail_000000465790_normalized.csv (행 10)
✅ service_list_sample.json → service_list_sample_normalized.json, service_list_sample_normalized.csv (행 10)
✅ support_conditions_000000465790.json → support_conditions_000000465790_normalized.json, support_conditions_000000465790_normalized.csv (행 10)
📦 합본 저장: support_conditions_all_normalized.(json/csv) — 총 30행
